# TD4 Advanced Regressions

#### Following this beautiful publication on Kaggle competition : 
https://www.kaggle.com/shaygu/house-prices-begginer-top-7/notebook

In [2]:
import github_command as gt

In [4]:
gt.push(file_to_transfer="TD4_Advanced_Regressions.ipynb", 
        message="initial commit on TD4", 
        repos="TDs_ESILV")

## TOC:
* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)

## First Bullet Header <a class="anchor" id="first-bullet"></a>